In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
import time
import pandas as pd

In [4]:
#webdriver取得
#driver = webdriver.Chrome()
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

#Google_search_keyword.csvを読み込む
keywords_list = pd.read_csv('Google_search_keyword.csv')
#キーワードを取得
keywords = keywords_list['keyword']

#リスト作成
df_list = []  #取得データ格納用

#Google_search_keyword.csvのキーワードで検索し情報を取得する
for keyword in keywords:

    #検索する
    url = f'https://www.google.com/search?q={keyword}'
    driver.get(url)
    time.sleep(2)
    
    #記事の一覧を取得する
    article_list = driver.find_elements_by_class_name('tF2Cxc')
    
    #記事ごとのURLを取得する
    url_list = []   #URL
    for article in article_list:
        url = article.find_element_by_tag_name('a').get_attribute('href')
        url_list.append(url)

    #リスト作成
    index_list = []             #インデックス
    title_list = []               #タイトル
    description_list = []    #description
    keywords_list = []      #keywords
    keyword_list = []        #キーワード
    text_num_list = []       #記事本文の文字数
    images_num_list = []  #画像数
    h1_tag_list = []          #h1タグ
    h1_tag_num_list = []  #h1タグ数
    h2_tag_list = []          #h2タグ
    h2_tag_num_list = []  #h2タグ数
    h3_tag_list = []          #h3タグ
    h3_tag_num_list = []  #h3タグ数
    h4_tag_list = []          #h4タグ
    h4_tag_num_list = []  #h4タグ数
    h5_tag_list = []          #h5タグ
    h5_tag_num_list = []  #h5タグ

    #各記事にアクセスし情報を取得する
    for i, url in enumerate(url_list):
        driver.get(url)
        time.sleep(2)
        
        #インデックス
        index = i + 1
        index_list.append(index)

        #タイトル
        title_list.append(driver.title)
        
        #description
        head_tag = driver.find_element_by_tag_name('head')
        try:
            description = head_tag.find_element_by_name('description')
            description = description.get_attribute('content')
            description_list.append(description)
        except:
            description_list.append('')
            
        #keywords
        try:
            keywords = head_tag.find_element_by_name('keywords')
            keywords = keywords.get_attribute('content')
            keywords_list.append(keywords)
        except:
            keywords_list.append('')
        
        #検索キーワード
        keyword_list.append(keyword)

        #記事本文の文字数
        body = driver.find_element_by_tag_name('body').text
        text_num_list.append(len(body))

        #画像数
        images = driver.find_elements_by_tag_name('img')
        images_num_list.append(len(images))

        #h1タグ
        h1_list = driver.find_elements_by_tag_name('h1')
        h1_tag = []
        for h1 in h1_list:
            h1 = h1.text
            h1 = h1.replace('\n', ' ')
            h1_tag.append(h1)
        h1_tag_list.append(h1_tag)

        #h1タグ数
        h1_tag_num_list.append(len(h1_tag))

        #h2タグ
        h2_list = driver.find_elements_by_tag_name('h2')
        h2_tag = []
        for h2 in h2_list:
            h2 = h2.text
            h2 = h2.replace('\n', ' ')
            h2_tag.append(h2)
        h2_tag_list.append(h2_tag)

        #h2タグ数
        h2_tag_num_list.append(len(h2_tag))

        #h3タグ
        h3_list = driver.find_elements_by_tag_name('h3')
        h3_tag = []
        for h3 in h3_list:
            h3 = h3.text
            h3 = h3.replace('\n', ' ')
            h3_tag.append(h3)
        h3_tag_list.append(h3_tag)

        #h3タグ数
        h3_tag_num_list.append(len(h3_tag))

        #h4タグ数
        h4_list = driver.find_elements_by_tag_name('h4')
        h4_tag = []
        for h4 in h4_list:
            h4 = h4.text
            h4 = h4.replace('\n', ' ')
            h4_tag.append(h4)
        h4_tag_list.append(h4_tag)

        #h4タグ数
        h4_tag_num_list.append(len(h4_tag))

        #h5タグ
        h5_list = driver.find_elements_by_tag_name('h5')
        h5_tag = []
        for h5 in h5_list:
            h5 = h5.text
            h5 = h5.replace('\n', ' ')
            h5_tag.append(h5)
        h5_tag_list.append(h5_tag)

        #h5タグ数
        h5_tag_num_list.append(len(h5_tag))

    #DataFrame作成
    df = pd.DataFrame([index_list, keyword_list, title_list, url_list, description_list, keywords_list, text_num_list, images_num_list, h1_tag_list, h1_tag_num_list,  h2_tag_list, h2_tag_num_list,  h3_tag_list, h3_tag_num_list,  h4_tag_list, h4_tag_num_list,  h5_tag_list, h5_tag_num_list]).T
    df.columns = ['index', 'keyword', 'title', 'url', 'description_list', 'keywords_list', 'char_num', 'images_num', 'h1_list', 'h1_num', 'h2_list', 'h2_num', 'h3_list', 'h3_num', 'h4_list', 'h4_num', 'h5_list', 'h5_num']
    df_list.append(df)

#カラムと取得したデータを結合
df_all = pd.concat(df_list)

driver.quit()

#CSV出力
df_all.to_csv('Google_search_result.csv', index = False)

#TSV出力
df_all.to_csv('Google_search_result.tsv', sep = '\t', index = False)